In [1]:
import os
import cv2
import sys
import dlib
import glob
import math
import random
import imageio

import numpy as np
import pandas as pd
import matplotlib

from imutils import face_utils
from matplotlib.pyplot import imshow
from IPython.display import Image, HTML, display
from matplotlib import pyplot as plt

print(sys.executable)
# print(os.sys.path)

ModuleNotFoundError: No module named 'dlib'

In [2]:
# print(os.listdir("../input/recognizing-faces-in-the-wild/test"))
all_faces = os.listdir("../input/recognizing-faces-in-the-wild/test")

face_1 = "../input/recognizing-faces-in-the-wild/test/" + all_faces[3]
face_2 = "../input/recognizing-faces-in-the-wild/test/" + all_faces[6]
face_3 = "../input/recognizing-faces-in-the-wild/test/" + all_faces[9]
face_4 = "../input/recognizing-faces-in-the-wild/test/" + all_faces[13]

Image(filename=face_4)

NameError: name 'Image' is not defined

In [3]:
print(os.listdir("../input/utilities/"))

# Bounding Box Frontal Face Detector
util_box = "../input/utilities/haarcascade_frontalface_default.xml"

# 5 Facial Points
predictor_path_5 = "../input/utilities/shape_predictor_5_face_landmarks.dat"

# 68 Facial Points 
predictor_path_68 = "../input/utilities/shape_predictor_68_face_landmarks.dat"

# 81 Facial Points 
predictor_path_81 = "../input/utilities/shape_predictor_81_face_landmarks.dat"

['shape_predictor_68_face_landmarks.dat', 'haarcascade_frontalface_default.xml', 'shape_predictor_81_face_landmarks.dat', 'shape_predictor_5_face_landmarks.dat']


In [4]:
# FUNCTION: Facial Bounding Box Detection
# https://realpython.com/face-recognition-with-python/

def bounding_box(imagePath, scale_factor, *mode):

    # Haar Cascade For Front
    faceCascade = cv2.CascadeClassifier(util_box)
    
    image = cv2.imread(imagePath)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Detect faces in the image
    faces = faceCascade.detectMultiScale(
        gray,
        scaleFactor=scale_factor,
        minNeighbors=5,
        minSize=(30, 30),
        flags = cv2.CASCADE_SCALE_IMAGE # cv2.CV_HAAR_SCALE_IMAGE
    )

    for (x, y, w, h) in faces:
        cv2.rectangle(image, (x, y), (x+w, y+h), (0, 255, 0), 2)
    
    if mode[0] == "test":
        cv2.imwrite('./results/result_bounding_box.png', image)
        print(faces)
    
    return faces

# TEST IT
bounding_box(face_1, 1.2, "test")
Image(filename='./results/result_bounding_box.png')

[[ 17   5 178 178]]


NameError: name 'Image' is not defined

In [5]:
def get_5_landmarks(face_path, *mode):
    
    detector = dlib.get_frontal_face_detector()
    predictor = dlib.shape_predictor(predictor_path_5)
    image = dlib.load_rgb_image(face_path)
    dets = detector(image, 1)

    faces = dlib.full_object_detections()
    
    for detection in dets:
        shape = predictor(image, detection)
        faces.append(shape)
        shape = face_utils.shape_to_np(shape)    
        
    # Cropped and aligned image
    image = dlib.get_face_chip(image, faces[0])
    image = cv2.resize(image,(500, 500))
    
    dets = detector(image, 1)
    shape = predictor(image, dets[0])
    shape = face_utils.shape_to_np(shape) 
    
    for (x, y) in shape:
        cv2.circle(image, (x, y), 2, (0, 255, 0), -1)

    if mode[0] == "test":
        cv2.imwrite('./results/result_5_points.png', cv2.cvtColor(image, cv2.COLOR_RGB2BGR))
        print(shape)
    elif mode[0] == "crop":
        cv2.imwrite('./test_faces/crop_.png', cv2.cvtColor(image, cv2.COLOR_RGB2BGR))
    
    return shape
  
get_5_landmarks(face_2, 'test')
# Image(filename="./test_faces/crop_alt_2.png")
Image(filename="./results/result_5_points.png")

NameError: name 'dlib' is not defined

In [6]:
def get_68_landmarks(face_path, *mode):

    detector = dlib.get_frontal_face_detector()
    predictor = dlib.shape_predictor(predictor_path_68)
    image = dlib.load_rgb_image(face_path)
    
    dets = detector(image, 1)
          
    for k, d in enumerate(dets):

        shape = predictor(image, d)
        shape = face_utils.shape_to_np(shape)

        for (x, y) in shape:
            cv2.circle(image, (x, y), 2, (0, 255, 0), -1)
        
        if mode[0] == "test":
            cv2.imwrite('./results/result_68_points.png', cv2.cvtColor(image, cv2.COLOR_RGB2BGR))
       
        return shape

get_68_landmarks(face_3, "test")
Image(filename='./results/result_68_points.png')

NameError: name 'dlib' is not defined

In [7]:
def get_delaunay(face_path):
    
    image = cv2.imread(face_path)
    size = image.shape
    rect = (0, 0, size[1], size[0])
    
    subdiv = cv2.Subdiv2D(rect);
    
    points = get_68_landmarks(face_path, None)
    points = list(points)

    for p in points:
        subdiv.insert(tuple(p))
    
    triangleList = subdiv.getTriangleList();

    return triangleList

In [8]:
# Check if a point is inside a rectangle
def rect_contains(rect, point) :
    if point[0] < rect[0] :
        return False
    elif point[1] < rect[1] :
        return False
    elif point[0] > rect[2] :
        return False
    elif point[1] > rect[3] :
        return False
    return True

def draw_delaunay(face_path, triangleList):
    
    image = cv2.imread(face_path)
    
    r = (0, 0, image.shape[1], image.shape[0])

    for t in triangleList :
         
        pt1 = (t[0], t[1])
        pt2 = (t[2], t[3])
        pt3 = (t[4], t[5])
         
        if rect_contains(r, pt1) and rect_contains(r, pt2) and rect_contains(r, pt3) :
         
            cv2.line(image, pt1, pt2, (255, 255, 255), 1, cv2.CV_AA, 0)
            cv2.line(image, pt2, pt3, (255, 255, 255), 1, cv2.CV_AA, 0)
            cv2.line(image, pt3, pt1, (255, 255, 255), 1, cv2.CV_AA, 0)
            
            cv2.circle(image, pt1, 2, (0, 0, 255), cv2.cv.CV_FILLED, cv2.CV_AA, 0)
            cv2.circle(image, pt2, 2, (0, 0, 255), cv2.cv.CV_FILLED, cv2.CV_AA, 0)
            cv2.circle(image, pt3, 2, (0, 0, 255), cv2.cv.CV_FILLED, cv2.CV_AA, 0)
    
    cv2.imwrite('./results/result_tri.png', image)

draw_delaunay(test_crop_uma, get_delaunay(test_crop_uma))
Image(filename="./results/result_tri.png")

NameError: name 'test_crop_uma' is not defined

In [9]:
def draw_voronoi(face_path):
 
    image = cv2.imread(face_path)
    rect = (0, 0, image.shape[1], image.shape[0])
    img_voronoi = np.zeros(image.shape, dtype = image.dtype)

    subdiv = cv2.Subdiv2D(rect)
    
    points = get_68_landmarks(face_path, None)
    points = list(points)

    for p in points:
        subdiv.insert(tuple(p))

    (facets, centers) = subdiv.getVoronoiFacetList([])
 
    for i in xrange(0,len(facets)) :
        ifacet_arr = []
        for f in facets[i] :
            ifacet_arr.append(f)
         
        ifacet = np.array(ifacet_arr, np.int)
        
#         color = (random.randint(0, 255), random.randint(0, 255), random.randint(0, 255))
        color = (255, 255, 255)
 
        cv2.fillConvexPoly(image, ifacet, color, cv2.CV_AA, 0);
        ifacets = np.array([ifacet])
        cv2.polylines(image, ifacets, True, (0, 0, 0), 1, cv2.CV_AA, 0)
        cv2.circle(image, (centers[i][0], centers[i][1]), 1, (0, 0, 0), cv2.cv.CV_FILLED, cv2.CV_AA, 0)

    cv2.imwrite('./results/result_voronoi_2.png', image)

draw_voronoi(test_crop_uma)
Image(filename='./results/result_voronoi_2.png')

NameError: name 'test_crop_uma' is not defined

In [10]:
img1 = cv2.imread("./results/result_voronoi_2.png")
img2 = cv2.imread("./results/result_tri.png")

# cv2.addWeighted(img1, alpha, img2, beta, gamma)
overlaid = cv2.addWeighted(img1, 0.5, img2, 0.5, 1)

cv2.imwrite("./results/result_points_overlay.png", overlaid)

Image(filename="./results/result_points_overlay.png")

NameError: name 'Image' is not defined

In [11]:
img1 = cv2.imread("./test_faces/crop_uma.png")
img2 = cv2.imread("./test_faces/crop_alt.png")
img3 = cv2.imread("./test_faces/crop_alt_2.png")

# cv2.addWeighted(img1, alpha, img2, beta, gamma)
overlaid = cv2.addWeighted(img1, 0.5, img3, 0.5, 1)

cv2.imwrite("./results/result_face_overlay.png", overlaid)

False

In [12]:
# Aligned & Overlaid (no triangulation (the stupid way))
Image(filename="./results/result_face_overlay.png")

NameError: name 'Image' is not defined

In [13]:
def get_avg_points(images, weights, mode):
#     print(mode + " Mode")

    indices = range(0, 67)
    columns = ['avg_x', 'avg_y', 'weighted_x', 'weighted_y']
    weightsum = sum(weights)
    cols_x = []
    cols_y = []  
    result = []
    
    for i in range(1, len(images) + 1):
        columns.append("f_" + str(i) +"_x")
        cols_x.append("f_" + str(i) +"_x")
        columns.append("f_" + str(i) +"_y")
        cols_y.append("f_" + str(i) +"_y")

    all_points = pd.DataFrame(index = indices, columns = columns)

    # Add Points To A Pandas Dataframe
    for i in range(0, len(images)):
        points = get_68_landmarks(images[i], None)
        for point in indices:
            all_points.iloc[point]["f_"+ str(i + 1) +"_x"] = points[point][0]
            all_points.iloc[point]["f_"+ str(i + 1) +"_y"] = points[point][1]

    # Get Average Coordinates
    for point in indices:
        xs = []
        ys = []

        for col in range(0, len(cols_x)): 
            xs.append(all_points.iloc[point][cols_x[col]])
            ys.append(all_points.iloc[point][cols_y[col]])

        avg_x = reduce(lambda x, y: x + y, xs) / float(len(xs))
        avg_y = reduce(lambda x, y: x + y, ys) / float(len(ys))

        all_points.iloc[point]['avg_x'] = avg_x
        all_points.iloc[point]['avg_y'] = avg_y
    
    # Get Weighted Average Coordinates
    for point in indices:
        
        xs = []
        ys = []
        wxs = []
        wys = []

        for col in range(0, len(cols_x)): 
            x = all_points.iloc[point][cols_x[col]]
            y = all_points.iloc[point][cols_y[col]]
            wt = weights[col]
            
            xs.append(x)
            ys.append(y)
            
            wxs.append(x * wt)
            wys.append(y * wt)
                    
        weighted_x = sum(wxs) / weightsum
        weighted_y = sum(wys) / weightsum
        
        all_points.iloc[point]['weighted_x'] = weighted_x
        all_points.iloc[point]['weighted_y'] = weighted_y
        
    if mode == 'test':   
        plt.xkcd()
        plt.figure(figsize=(20,20))
        for el in range(1, len(images) + 1): 
            plt.scatter(all_points["f_" + str(el) + "_x"], all_points["f_" + str(el) + "_y"], c='b', s=10, label='data') # [BLUE]
        plt.scatter(all_points["avg_x"], all_points["avg_y"], c='r', s=100, label='data') # AVERAGE POINTS   [RED]
        plt.scatter(all_points["weighted_x"], all_points["weighted_y"], c='y', s=100, label='data') # AVERAGE POINTS   [RED]
        plt.gca().invert_yaxis()
        plt.show()
    if mode == 'weighted':
        for i in indices:
            result.append((all_points.iloc[i]["weighted_x"], all_points.iloc[i]["weighted_y"],))
        return result     
    if mode == 'average':
        for i in indices:
            result.append((all_points.iloc[i]["avg_x"], all_points.iloc[i]["avg_y"],))
        return result


# get_avg_points([ "./test_faces/crop_3_1.png", 
#                  "./test_faces/crop_3_2.png", 
#                  "./test_faces/crop_3_3.png", 
#                  "./test_faces/crop_3_4.png", 
#                  "./test_faces/crop_3_5.png" ], None, "average")

get_avg_points([ "./test_faces/crop_3_1.png", 
                 "./test_faces/crop_3_2.png", 
                 "./test_faces/crop_3_3.png", 
                 "./test_faces/crop_3_4.png", 
                 "./test_faces/crop_3_5.png" ], [6, 8, 9, 4, 9], "test")

NameError: name 'pd' is not defined

In [14]:
image = cv2.imread("./results/result_face_overlay.png")

for row in range(0, face_frame.shape[0]):
    cv2.circle(image, (face_frame.iloc[row]["f_1_x"], face_frame.iloc[row]["f_1_y"]), 2, (255, 0, 0), -1)
    cv2.circle(image, (face_frame.iloc[row]["f_2_x"], face_frame.iloc[row]["f_2_y"]), 2, (0, 0, 255), -1)
    cv2.circle(image, (face_frame.iloc[row]["avg_x"], face_frame.iloc[row]["avg_y"]), 2, (0, 255, 0), -1)
    
cv2.imwrite("./results/result_face_overlay_points.png", image)

Image(filename="./results/result_face_overlay_points.png")

NameError: name 'face_frame' is not defined

In [15]:
# Apply affine transform calculated using srcTri and dstTri to src and output an image of size.
def applyAffineTransform(src, srcTri, dstTri, size):
    
    # Given a pair of triangles, find the affine transform.
    warpMat = cv2.getAffineTransform(np.float32(srcTri), np.float32(dstTri))
    
    # Apply the Affine Transform just found to the src image
    dst = cv2.warpAffine(src, warpMat, (size[0], size[1]), None, flags=cv2.INTER_LINEAR, borderMode=cv2.BORDER_REFLECT_101)

    return dst


# Warps and alpha blends triangular regions from img1 and img2 to img
def morphTriangle(img1, img2, img, t1, t2, t, alpha):

    # Find bounding rectangle for each triangle
    r1 = cv2.boundingRect(np.float32([t1]))
    r2 = cv2.boundingRect(np.float32([t2]))
    r = cv2.boundingRect(np.float32([t]))

    # Offset points by left top corner of the respective rectangles
    t1Rect = []
    t2Rect = []
    tRect = []

    for i in range(0, 3):
        tRect.append(((t[i][0] - r[0]),(t[i][1] - r[1])))
        t1Rect.append(((t1[i][0] - r1[0]),(t1[i][1] - r1[1])))
        t2Rect.append(((t2[i][0] - r2[0]),(t2[i][1] - r2[1])))

    # Get mask by filling triangle
    mask = np.zeros((r[3], r[2],3), dtype = np.float32)
    cv2.fillConvexPoly(mask, np.int32(tRect), (1.0, 1.0, 1.0), 16, 0);

    # Apply warpImage to small rectangular patches
    img1Rect = img1[r1[1]:r1[1] + r1[3], r1[0]:r1[0] + r1[2]]
    img2Rect = img2[r2[1]:r2[1] + r2[3], r2[0]:r2[0] + r2[2]]

    size = (r[2], r[3])
    warpImage1 = applyAffineTransform(img1Rect, t1Rect, tRect, size)
    warpImage2 = applyAffineTransform(img2Rect, t2Rect, tRect, size)

    # Alpha blend rectangular patches
    imgRect = (1.0 - alpha) * warpImage1 + alpha * warpImage2
    
    # Copy triangular region of the rectangular patch to the output image
    img[r[1]:r[1]+r[3], r[0]:r[0]+r[2]] = img[r[1]:r[1]+r[3], r[0]:r[0]+r[2]] * ( 1 - mask ) + imgRect * mask


def make_morph(file1, file2, frames):

    img1 = cv2.imread(file1)
    img2 = cv2.imread(file2)

    img1 = np.float32(img1)
    img2 = np.float32(img2)
    imgx = np.float32(cv2.imread('./test_faces/blank500.png')) # blank image

    points1 = get_68_landmarks(file1, None)
    points2 = get_68_landmarks(file2, None)
    
    filenames = []
    
    for frame in range(0, frames):
        
        points3 = [] # weighted avg points
        
        alpha = float(1 / float(frames)) * float(frame)

        for i in range(0, len(points1)):
            x = ( 1 - alpha ) * points1[i][0] + alpha * points2[i][0]
            y = ( 1 - alpha ) * points1[i][1] + alpha * points2[i][1]
            points3.append((int(x),int(y)))

        imgMorph = np.zeros(img1.shape, dtype = img1.dtype)

        rect = (0, 0, img1.shape[1], img1.shape[0])

        subdiv = cv2.Subdiv2D(rect);

        for p in points3:
            subdiv.insert(p)

        triangleList = subdiv.getTriangleList();

        for line in triangleList:

            x = [ int(line[0]), int(line[1]) ]
            y = [ int(line[2]), int(line[3]) ]
            z = [ int(line[4]), int(line[5]) ]

            if rect_contains(rect, x) and rect_contains(rect, y) and rect_contains(rect, z):
                
                i_3_x = points3.index((x[0], x[1]))
                i_3_y = points3.index((y[0], y[1]))
                i_3_z = points3.index((z[0], z[1]))

                t1 = [ points1[i_3_x], points1[i_3_y], points1[i_3_z] ]
                t2 = [ points2[i_3_x], points2[i_3_y], points2[i_3_z] ]
                t3 = [ points3[i_3_x], points3[i_3_y], points3[i_3_z] ] # Weighted Avg Point Triangle

                morphTriangle(img1, img2, imgMorph, t1, t2, t3, alpha)
        
        filenames.append("./results/gif/result_face_merge_" + str(frame) + ".png")
        cv2.imwrite("./results/gif/result_face_merge_" + str(frame) + ".png", imgMorph)

    return filenames

# morph_files = make_morph(test_crop_uma, test_crop_alt_2, 1)
morph_files = make_morph('./test_faces/crop_3_1.png', './test_faces/crop_3_2.png', 10)

images = []
filenames = morph_files

for filename in filenames:
    images.append(imageio.imread(filename)) 

print(len(images))

imageio.mimsave('./results/gif/movie.gif', images)



NameError: name 'np' is not defined

In [16]:
HTML('<img id="moviegif" src="./results/gif/movie.gif">')

NameError: name 'HTML' is not defined

In [17]:
img_1 = './test_faces/crop_3_1.png'
img_2 = './test_faces/crop_3_2.png'
img_3 = './test_faces/crop_3_3.png'
img_4 = './test_faces/crop_3_4.png'
img_5 = './test_faces/crop_3_5.png'

filenames = [ img_1, img_2, img_3, img_4, img_5 ]

weights = [1, 1, 10, 1, 1] # the 1-10 scale

Image(filename="./test_faces/weightedmergeexamples.jpg")

NameError: name 'Image' is not defined

In [18]:
def make_weighted_combo(filenames, weights):

    filecount = len(filenames)
    
    result_files = []
    
    avg_points = get_avg_points(filenames, weights, 'weighted')

    imgx = np.float32(cv2.imread('./test_faces/blank500.png', cv2.IMREAD_UNCHANGED)) # blank image

    print(weights, filecount)
    
    for i in range(0, len(filenames)):
        
        print(i)
        
        file = filenames[i]
        weight = float(weights[i]) / float(sum(weights))
        
        print(file, weight) 
        
        img = cv2.imread(file) # , cv2.IMREAD_UNCHANGED
        img = np.float32(img)
        
        print(type(img))

        points = get_68_landmarks(file, None)

        imgMorph = np.zeros(img.shape, dtype = img.dtype)

        rect = (0, 0, img.shape[1], img.shape[0])
        subdiv = cv2.Subdiv2D(rect);

        for p in avg_points:
            subdiv.insert(p)

        triangleList = subdiv.getTriangleList();

        for line in triangleList:

            x = [ int(line[0]), int(line[1]) ]
            y = [ int(line[2]), int(line[3]) ]
            z = [ int(line[4]), int(line[5]) ]

            if rect_contains(rect, x) and rect_contains(rect, y) and rect_contains(rect, z):

                i_3_x = avg_points.index((x[0], x[1]))
                i_3_y = avg_points.index((y[0], y[1]))
                i_3_z = avg_points.index((z[0], z[1]))

                t1 = [ points[i_3_x], points[i_3_y], points[i_3_z] ]
                ta = [ avg_points[i_3_x], avg_points[i_3_y], avg_points[i_3_z] ]
                
                # Find bounding rectangle for each triangle
                r1 = cv2.boundingRect(np.float32([t1]))
                ra = cv2.boundingRect(np.float32([ta]))

                # Offset points by left top corner of the respective rectangles
                t1Rect = []
                taRect = []

                for c in range(0, 3):
                    taRect.append(((ta[c][0] - ra[0]),(ta[c][1] - ra[1])))
                    t1Rect.append(((t1[c][0] - r1[0]),(t1[c][1] - r1[1])))

                # Get mask by filling triangle
                mask = np.zeros((ra[3], ra[2], 3), dtype = np.float32)
                cv2.fillConvexPoly(mask, np.int32(taRect), (1.0, 1.0, 1.0), 16, 0);

                # Apply warpImage to small rectangular patches
                img1Rect = img[r1[1]:r1[1] + r1[3], r1[0]:r1[0] + r1[2]]

                size = (ra[2], ra[3])
                warpImage1 = applyAffineTransform(img1Rect, t1Rect, taRect, size)

                # Alpha blend rectangular patches
                imgRect = warpImage1 # * weight
                
                # Copy triangular region of the rectangular patch to the output image
                imgMorph[ra[1]:ra[1]+ra[3], ra[0]:ra[0]+ra[2]] = imgMorph[ra[1]:ra[1]+ra[3], ra[0]:ra[0]+ra[2]] * ( 1 - mask ) + imgRect * mask
        
        # Remove Black Background
        tmp = cv2.cvtColor(imgMorph, cv2.COLOR_BGR2GRAY)
        _,alpha = cv2.threshold(tmp, 0, 255, cv2.THRESH_BINARY)
        b, g, r = cv2.split(imgMorph)
        rgba = [b,g,r, alpha]
        dst = cv2.merge(rgba, 4)
        imgMorph = dst
        
        # Make transparent 
        blank = cv2.imread('./test_faces/blank500.png', cv2.IMREAD_UNCHANGED)
        alpha = weight
        imgMorph = cv2.addWeighted(imgMorph, alpha, blank, 1 - alpha, 0, blank, dtype=cv2.CV_64F)
        
        # Add to result image
        imgx = cv2.add(imgx, imgMorph, dtype=cv2.CV_64F)
        
        result_files.append("./results/gif/result_multi_face_merge" + str(i) + ".png")   
        cv2.imwrite("./results/gif/result_multi_face_merge" + str(i) + ".png", imgMorph)
    
    print(result_files)
    ratingStr = str(weights[0]) + '-' + str(weights[1]) + '-' + str(weights[2]) + '-' + str(weights[3]) + '-' + str(weights[4])
    cv2.imwrite("./results/gif/result_merge_" + ratingStr + ".png", imgx)

make_weighted_combo(filenames, weights)

NameError: name 'pd' is not defined

In [19]:
ratings = '10-1-1-10-1'
print('ratings: ', ratings)
Image(filename='./results/gif/result_merge_' + ratings + '.png')

ratings:  10-1-1-10-1


NameError: name 'Image' is not defined

In [20]:
ratings = '1-10-5-1-5'
print('ratings: ', ratings)
Image(filename='./results/gif/result_merge_' + ratings + '.png')

ratings:  1-10-5-1-5


NameError: name 'Image' is not defined

In [21]:
ratings = '1-1-1-1-1'
print('ratings: ', ratings)
Image(filename='./results/gif/result_merge_' + ratings + '.png')

ratings:  1-1-1-1-1


NameError: name 'Image' is not defined

In [22]:
ratings = '1-1-10-1-1'
print('ratings: ', ratings)
Image(filename='./results/gif/result_merge_' + ratings + '.png')

ratings:  1-1-10-1-1


NameError: name 'Image' is not defined

In [23]:
# TODO:
# Age Estimation: https://github.com/spmallick/learnopencv
# Gender Estimation: https://github.com/spmallick/learnopencv
# Hair Color
# Eye Color
# Facial Hair
# Piercings
# Scars
# Real / Spoofed Faces
# Multi-face merge
# function for finding only the most directly aligned 68 points
